# COI Data Preparation: Refinement and Development

**Objective:** Refine the data preparation pipeline for the COI gene, the primary barcode for animal identification, using the BOLD database.

**Methodology:**
1.  Work from a small, manageable sample of the full BOLD database.
2.  Develop a new taxonomy parser specifically designed for the pipe-separated (`|`) format of BOLD FASTA headers.
3.  Apply the full data cleaning and feature engineering workflow.
4.  Save the final, uniquely named artifacts for the COI pipeline.

In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm.auto import tqdm
from pathlib import Path
import sys

# Set up project path
project_root = Path.cwd().parent

# --- Verification Step ---
print(f"Project Root: {project_root}")

Project Root: C:\Users\jampa\Music\atlas-v3


In [3]:
# --- Define Core Directories ---
RAW_DATA_DIR = project_root / "data" / "raw"
PROCESSED_DATA_DIR = project_root / "data" / "processed"
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

# --- Define COI Specific File Paths ---

# --- FIX: Use the exact filename of the downloaded BOLD database ---
# Path to the full, original BOLD database file
FULL_BOLD_PATH = RAW_DATA_DIR / "BOLD_Public.29-Aug-2025.fasta"

# Path to the small sample file we will create for development
SAMPLE_BOLD_PATH = RAW_DATA_DIR / "BOLD_sample_10k.fasta"

# --- Verification Step ---
if FULL_BOLD_PATH.exists():
    print("Source BOLD database found.")
    print(f"  - Location: {FULL_BOLD_PATH}")
else:
    print(f"ERROR: The source BOLD database was not found at the expected location.")
    print(f"  - Expected: {FULL_BOLD_PATH}")
    print("Please ensure the file is downloaded and correctly named in the 'data/raw' directory.")

Source BOLD database found.
  - Location: C:\Users\jampa\Music\atlas-v3\data\raw\BOLD_Public.29-Aug-2025.fasta


In [4]:
# =============================================================================
# STEP 1: CREATE A DEVELOPMENT SAMPLE FROM BOLD DATA
# =============================================================================
#
# OBJECTIVE:
#   To create a smaller 10,000-sequence sample from the full BOLD database.
#
# RATIONALE:
#   Working with a smaller sample allows for rapid, interactive development
#   and debugging of the subsequent parsing and cleaning steps without the
#   long processing times required for the full dataset. This script is
#   designed to be run only once; if the sample file is found, this
#   step will be skipped.
#
# =============================================================================

# --- Configuration ---
SAMPLE_SIZE = 10000

# --- Main Logic ---
# This check prevents us from re-running this process unnecessarily.
if not SAMPLE_BOLD_PATH.exists():
    print(f"Creating a sample of {SAMPLE_SIZE} sequences from the BOLD database...")
    
    sample_records = []
    # Open the full BOLD database file for reading
    with open(FULL_BOLD_PATH, "r") as handle_in:
        # Create an efficient iterator over the FASTA records
        records_iterator = (record for record in SeqIO.parse(handle_in, "fasta"))
        
        # Use tqdm to show a progress bar while iterating
        for i, record in tqdm(enumerate(records_iterator), total=SAMPLE_SIZE, desc="Sampling BOLD DB"):
            # Stop once we have collected the desired number of samples
            if i >= SAMPLE_SIZE:
                break
            sample_records.append(record)
            
    # Write the collected records to our new sample file
    with open(SAMPLE_BOLD_PATH, "w") as handle_out:
        SeqIO.write(sample_records, handle_out, "fasta")
        
    print(f"\nSuccessfully created sample file with {len(sample_records)} sequences.")
    print(f"   Location: {SAMPLE_BOLD_PATH}")
else:
    print(f"BOLD sample file already exists. No action needed.")
    print(f"   Location: {SAMPLE_BOLD_PATH}")

Creating a sample of 10000 sequences from the BOLD database...


Sampling BOLD DB:   0%|          | 0/10000 [00:00<?, ?it/s]


Successfully created sample file with 10000 sequences.
   Location: C:\Users\jampa\Music\atlas-v3\data\raw\BOLD_sample_10k.fasta


In [6]:
# =============================================================================
# STEP 2 (REVISED): PARSE TAXONOMY FROM BOLD SAMPLE DATA
# =============================================================================
#
# OBJECTIVE:
#   To use a new, more robust parser to correctly handle the complex and
#   inconsistent BOLD database FASTA header format.
#
# RATIONALE:
#   Initial analysis revealed the taxonomy is often a comma-separated string
#   within a larger pipe-separated header. This new v2 parser is designed
#   to specifically find and process this nested format, ensuring accurate
#   extraction of taxonomic ranks.
#
# =============================================================================

# --- A list to hold our structured data ---
parsed_data = []

# --- Define the new, more robust BOLD-specific parsing function ---
def parse_bold_taxonomy_v2(description):
    """
    Parses the complex, pipe-and-comma-separated BOLD FASTA header.
    """
    # Initialize a dictionary with all ranks set to None
    parsed_ranks = {
        'kingdom': None, 'phylum': None, 'class': None, 'order': None,
        'family': None, 'genus': None, 'species': None
    }
    
    try:
        # First, split the entire description by the pipe character
        parts = description.split('|')
        
        # Now, find the part that actually contains the taxonomy.
        # We'll assume it's the part with commas and "Animalia".
        taxonomy_str = ""
        for part in parts:
            if ',' in part and 'Animalia' in part:
                taxonomy_str = part
                break
        
        # If we found a valid taxonomy string, process it
        if taxonomy_str:
            ranks = taxonomy_str.split(',')
            
            # Assign ranks based on their position in the comma-separated list
            if len(ranks) > 0: parsed_ranks['kingdom'] = ranks[0]
            if len(ranks) > 1: parsed_ranks['phylum'] = ranks[1]
            if len(ranks) > 2: parsed_ranks['class'] = ranks[2]
            if len(ranks) > 3: parsed_ranks['order'] = ranks[3]
            if len(ranks) > 4: parsed_ranks['family'] = ranks[4]
            if len(ranks) > 5: parsed_ranks['genus'] = ranks[5]
            if len(ranks) > 6: parsed_ranks['species'] = ranks[6]

    except Exception:
        # Pass silently if any unexpected error occurs
        pass
        
    return parsed_ranks

# --- Apply the new parser to our sample data ---
print("Applying revised BOLD taxonomy parser (v2) to sample data...")

# Loop through the records in our sample file
with open(SAMPLE_BOLD_PATH, "r") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta"), total=SAMPLE_SIZE, desc="Parsing BOLD headers"):
        # The entire description line contains the taxonomy
        taxonomy_dict = parse_bold_taxonomy_v2(record.description)
        
        # Store the essential sequence information
        taxonomy_dict['id'] = record.id
        taxonomy_dict['sequence'] = str(record.seq)
        
        parsed_data.append(taxonomy_dict)

# --- Create and Verify the DataFrame ---
# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(parsed_data)

print(f"\nParsing complete. Created a DataFrame with {len(df)} rows.")

# Display the first 5 rows for a preliminary check
print("\nPreview of the structured data (first 5 rows):")
display(df.head())

# Display 5 random rows to check for consistency across the dataset
print("\nPreview of the structured data (5 random rows):")
display(df.sample(5))

Applying revised BOLD taxonomy parser (v2) to sample data...


Parsing BOLD headers:   0%|          | 0/10000 [00:00<?, ?it/s]


Parsing complete. Created a DataFrame with 10000 rows.

Preview of the structured data (first 5 rows):


,kingdom,phylum,class,order,family,genus,species,id,sequence
0,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Oenochrominae,Arhodia,"AANIC003-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
1,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Oenochrominae,Arhodia,"AANIC101-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
2,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Oenochrominae,Arhodia,"AANIC102-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCTGGTATAATTGGAACT...
3,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Oenochrominae,Arhodia,"AANIC104-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCAGGTATAATTGGAACT...
4,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Oenochrominae,Arhodia,"AANIC105-10|COI-5P|Australia|Animalia,Arthropo...",AACATTATATTTTATTTTTGGTATTTGAGCAGGTATAATTGGAACT...



Preview of the structured data (5 random rows):


,kingdom,phylum,class,order,family,genus,species,id,sequence
8283,Animalia,Arthropoda,Insecta,Diptera,Dolichopodidae,None,None,"AACTA6487-20|COI-5P|Australia|Animalia,Arthrop...",CATTATATTTTATTTTTGGAGCATGAGCAGGTATAGTTGGTACATC...
6151,Animalia,Arthropoda,Insecta,Hymenoptera,Thynnidae,None,None,"AACTA4384-20|COI-5P|Australia|Animalia,Arthrop...",CTTTATTTTATTTTCGCTATTTGAGCTGGATTAATAGGAACATCAA...
8288,Animalia,Arthropoda,Insecta,Diptera,Phoridae,None,None,"AACTA6591-20|COI-5P|Australia|Animalia,Arthrop...",ATTATATTTTATTTTTGGTGCTTGAGCAGGAATAGTAGGAACTTCA...
5337,Animalia,Arthropoda,Insecta,Diptera,Hybotidae,None,None,"AACTA3416-20|COI-5P|Australia|Animalia,Arthrop...",TCTTTATTTTATATTTGGTGCCTGAGCCGGAATAGTAGGAACATCA...
6459,Animalia,Arthropoda,Insecta,Hymenoptera,Bethylidae,None,None,"AACTA4703-20|COI-5P|Australia|Animalia,Arthrop...",ATATTATATTTTGTATTTAGAATATGATCAGGTATAATTGGATCAT...
